## There are three sections here. See instructions below.
Section 1 should be run using qiskit-terra master. This will cause standard compilation of RX(theta) into pulses.

Section 2 should be run using Pranav's qiskit-terra fork, which will compile to a direct RX(theta) rotation. So after running Section 1, switch to the qiskit-terra fork, and restart the kernel.

Section 3 is data analysis, which can be run with either qiskit-terra version. Easiest to keep running same fork, no need to restart kernel.

In [ ]:
qubit_num = 0  # or change to a qubit that has bad coherence

---
## Section 1: Standard Compilation (run with qiskit-terra master)

In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk
q.IBMQ.providers()    # List all available providers

In [ ]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
num_angles = 17  # will exclude endpoints of 0 and 2pi
angles = np.linspace(0, np.pi, num_angles+1, endpoint=False)[1:]  # get rid of endpoints
print(angles)

In [ ]:
schedules = []
for angle in angles:
    schedule = cmd_def.get('u3', [qubit_num], P0=angle, P1=3*np.pi/2, P2=np.pi/2)  # RX(angle) = U3(angle, 3pi/2, pi/2)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [ ]:
# sanity check, show a schedule
schedules[10].draw(plot_range=(0, 1500))

In [ ]:
shots = 1000
standard_job = q.execute(schedules, backend=backend, shots=shots)
print(standard_job.job_id())

------
## Section 2: Direct RX (Switch to Pranav's qiskit-terra fork, then restart kernel but don't clear output)

In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk
q.IBMQ.providers()    # List all available providers

In [ ]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
num_angles = 17  # will exclude endpoints of 0 and 2pi
angles = np.linspace(0, np.pi, num_angles+1, endpoint=False)[1:]  # get rid of endpoints
print(angles)

In [ ]:
schedules = []
for angle in angles:
    circuit = q.QuantumCircuit(20)
    circuit.rx(angle, qubit_num)
    decomposed_circuit = circuit.decompose()
    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(circuit, backend, optimization_level=1)
    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [ ]:
# sanity check, show a schedule
schedules[10].draw(plot_range=(0, 1500))

In [ ]:
shots = 1000
direct_job = q.execute(schedules, backend=backend, shots=shots)
print(direct_job.job_id())

---
## Section 3: Data Analysis

In [ ]:
standard_job = backend.retrieve_job('PASTE JOB ID HERE')
direct_job = backend.retrieve_job('PASTE JOB ID HERE')

In [ ]:
standard_result = standard_job.result()
direct_result = direct_job.result()

In [ ]:
from qiskit.ignis.verification.tomography.data import marginal_counts

In [ ]:
standard_fracs = []
for i in range(num_angles):
    counts = marginal_counts(standard_result.get_counts(i), meas_qubits=[qubit_num])
    standard_fracs.append(counts.get('1', 0) / shots)
print(standard_fracs)

direct_fracs = []
for i in range(num_angles):
    counts = marginal_counts(direct_result.get_counts(i), meas_qubits=[qubit_num])
    direct_fracs.append(counts.get('1', 0) / shots)
print(direct_fracs)

In [ ]:
plt.scatter(angles, standard_fracs)
plt.scatter(angles, direct_fracs)
plt.plot(angles, [np.sin(angle / 2) ** 2 for angle in angles])
plt.axis([0, np.pi, 0, 1])